# Welcome to my kernel 
Skin cancer is the most common human malignancy, is primarily diagnosed visually, beginning with an initial clinical screening and followed potentially by dermoscopic analysis, a biopsy and histopathological examination. Automated classification of skin lesions using images is a challenging task owing to the fine-grained variability in the appearance of skin lesions.

This the **HAM10000 ("Human Against Machine with 10000 training images")** dataset.It consists of 10015 dermatoscopicimages which are released as a training set for academic machine learning purposes and are publiclyavailable through the ISIC archive. This benchmark dataset can be used for machine learning and for comparisons with human experts. 

It has 7 different classes of skin cancer which are listed below :<br>
**1. Melanocytic nevi <br>**
**2. Melanoma <br>**
**3. Benign keratosis-like lesions <br>**
**4. Basal cell carcinoma <br>**
**5. Actinic keratoses <br>**
**6. Vascular lesions <br>**
**7. Dermatofibroma<br>**

In this kernel I will try to detect 7 different classes of skin cancer using Convolution Neural Network with keras tensorflow in backend and then analyse the result to see how the model can be useful in practical scenario.<br>
We will move step by step process to classify 7 classes of cancer.

In this kernel I have followed following 14 steps for model building and evaluation which are as follows : <br>
**Step 1 : Importing Essential Libraries**<br>
**Step 2: Making Dictionary of images and labels** <br>
**Step 3: Reading and Processing Data** <br>
**Step 4: Data Cleaning** <br>
**Step 5: Exploratory data analysis (EDA)** <br>
**Step 6: Loading & Resizing of images **<br>
**Step 7: Train Test Split**<br>
**Step 8: Normalization**<br>
**Step 9: Label Encoding** <br>
**Step 10: Train validation split** <br>
**Step 11: Model Building (CNN)** <br>
**Step 12: Setting Optimizer & Annealing** <br>
**Step 13: Fitting the model**<br>
**Step 14: Model Evaluation (Testing and validation accuracy, confusion matrix, analysis of misclassified instances)** <br>

<img src="https://image.ibb.co/n8PBkL/cover.png">

# Step 1 : importing Essential Libraries

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(123)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import *
import itertools

import keras
from keras.utils.np_utils import to_categorical # used for converting labels to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras import backend as K
import itertools
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.applications import VGG19, ResNet50
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

In [ ]:
#1. Function to plot model's validation loss and validation accuracy
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()


# Step 2 : Making Dictionary of images and labels
In this step I have made the image path dictionary by joining the folder path from base directory base_skin_dir and merge the images in jpg format from both the folders HAM10000_images_part1.zip and HAM10000_images_part2.zip

In [ ]:
base_skin_dir = os.path.join('..', 'input','skin-cancer-mnist-ham10000')

# Merging images from both folders HAM10000_images_part1.zip and HAM10000_images_part2.zip into one dictionary

imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.jpg'))}

# This dictionary is useful for displaying more human-friendly labels later on

lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

# Step 3 : Reading & Processing data

In this step we have read the csv by joining the path of image folder which is the base folder where all the images are placed named base_skin_dir.
After that we made some new columns which is easily understood for later reference such as we have made column path which contains the image_id, cell_type which contains the short name of lesion type and at last we have made the categorical column cell_type_idx in which we have categorize the lesion type in to codes from 0 to 6

In [ ]:


skin_df = pd.read_csv(os.path.join(base_skin_dir, 'HAM10000_metadata.csv'))

# Creating New Columns for better readability

skin_df['path'] = skin_df['image_id'].map(imageid_path_dict.get)
skin_df['cell_type'] = skin_df['dx'].map(lesion_type_dict.get) 
skin_df['cell_type_idx'] = pd.Categorical(skin_df['cell_type']).codes


In [ ]:
# Now lets see the sample of tile_df to look on newly made columns
skin_df.head()

# Step 4 : Data Cleaning
In this step we check for Missing values and datatype of each field 

In [ ]:
skin_df.isnull().sum()

As it is evident from the above that only age has null values which is 57 so we will fill the null values by their mean.

In [ ]:
skin_df['age'].fillna((skin_df['age'].mean()), inplace=True)

Now, lets check the presence of null values  again

In [ ]:
skin_df.isnull().sum()

In [ ]:
print(skin_df.dtypes)

# Step 5 : EDA
In this we will explore different features of the dataset , their distrubtions and actual counts

Plot to see distribution of 7 different classes of cell type

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize= (10, 5))
skin_df['cell_type'].value_counts().plot(kind='bar', ax=ax1)

Its seems from the above plot that in this dataset cell type Melanecytic nevi has very large number of instances in comparison to other cell types

Plotting of Technical Validation field (ground truth) which is dx_type to see the distribution of its 4 categories which are listed below :<br>
**1. Histopathology(Histo):**  Histopathologic diagnoses of excised lesions have been
performed by specialized dermatopathologists. <br>
**2. Confocal:** Reflectance confocal microscopy is an in-vivo imaging technique with a resolution at near-cellular level , and some facial benign with a grey-world assumption of all training-set images in Lab-color space before
and after  manual histogram changes.<br>
**3. Follow-up:** If nevi monitored by digital dermatoscopy did not show any changes during 3 follow-up visits or 1.5 years biologists  accepted this as evidence of biologic benignity. Only nevi, but no other benign diagnoses were labeled with this type of ground-truth because dermatologists usually do not monitor dermatofibromas, seborrheic keratoses, or vascular lesions. <br>
**4. Consensus:** For typical benign cases without histopathology or followup biologists  provide an expert-consensus rating of authors PT and HK. They applied the consensus label only if both authors independently gave the same unequivocal benign diagnosis. Lesions with this type of groundtruth were usually photographed for educational reasons and did not need
further follow-up or biopsy for confirmation.


In [ ]:
skin_df['dx_type'].value_counts().plot(kind='bar')

Plotting the distribution of localization field 

In [ ]:
skin_df['localization'].value_counts().plot(kind='bar')

It seems back , lower extremity,trunk and upper extremity are heavily compromised regions of skin cancer 

Now, check the distribution of Age

In [ ]:
skin_df['age'].hist(bins=15)

It seems that there are larger instances of patients having age from 30 to 60

Lets see the distribution of males and females

In [ ]:

skin_df['sex'].value_counts().plot(kind='bar')

Now lets visualize agewise distribution of skin cancer types

In [ ]:
ax = sns.boxplot('cell_type', 'age',data=skin_df)
plt.setp(ax.get_xticklabels(), rotation=90)

# Using metadata to predict the class 



In [ ]:
#.head()

In [ ]:
#skin_df2 = skin_df  # creates new database
#skin_df2['malign'] = 0 # adds a variable that will represent if the tumor is benign or not

In [ ]:
#skin_df2.head()

In [ ]:

#skin_df2.loc[skin_df['cell_type_idx']==1, 'malign'] = 1
#skin_df2.loc[skin_df['cell_type_idx']==5, 'malign'] = 1
#skin_df2.head()

## Let's make some graphs
Plot the different covariates against the target variable in order to see if we can some spot some kind of intereseting relationship

In [ ]:
#ax = sns.boxplot('malign', 'age',data=skin_df2)
#plt.setp(ax.get_xticklabels(), rotation=90)

 In the graph above we can see that people that were diagnosed with a malign cancer tend to be older. 
 
 Let's see now what happens when we study the relationship between the sex of the patients.


In [ ]:
#pd.crosstab(skin_df2.malign,skin_df2.sex).plot(kind='bar')#
#plt.title('Malign and benign lesions count')
#plt.ylabel('count')
#plt.savefig('mb_per_sex')

So we can see that in both cases female tend to be more affected with malign tumor. This could be related to the fact that women tend to live longer and, as we've seen before malign tumor affect older people.

In [ ]:
#malign_tumor = skin_df2[skin_df2.malign == 1]
#benign_tumor = skin_df2[skin_df2.malign == 0]
#malign_tumor.head()

In [ ]:

#malign_tumor['localization'].value_counts().plot(kind='bar')


In [ ]:
#malign_tumor['localization'].value_counts() / len(malign_tumor)#


We can see that the back is by far the most common location for a malign tumor.  As it almost makes up for almost one third of all the malign cases. 

In [ ]:
#benign_tumor['localization'].value_counts().plot(kind='bar')#

In [ ]:
#benign_tumor['localization'].value_counts() / len(benign_tumor)

Here we can see that the distribution is different from the malign one as, even though the most common locations are the same here we don't see a particular location sticking up as it did in the malign case.

In [ ]:
#skin_df2['localization'].value_counts().plot(kind='bar')

This concludes our EDA. From what've been able to see from these graphs is that there are some important characteristics that can hepl us identify a malign tumor from a benign one. LEt's see if we can than build a model that could help us in this classification process. 

In [ ]:
#skin_df2.head()#

In [ ]:
##features=skin_df2.drop(columns=['lesion_id', 'image_id', 'dx', 'dx_type', 'path', 'cell_type', 'cell_type_idx', 'malign'],axis=1)###
#target=skin_df2['malign']


In [ ]:
#target.head()

In [ ]:
#features.head()

In [ ]:
#dummies1 = pd.get_dummies(features['localization']).rename(columns=lambda x: str(x))
#dummies2 = pd.get_dummies(features['sex']).rename(columns = lambda x : str(x))

In [ ]:
#features = pd.concat([features, dummies1, dummies2], axis = 1)

#features.head()

In [ ]:
#features = features.drop(columns = ['localization', 'sex'])

In [ ]:
#x_train_lo, x_test_lo, y_train_lo, y_test_lo = train_test_split(features, target, test_size=0.20,random_state=2)

In [ ]:
#from sklearn import linear_model
#logistic = linear_model.LogisticRegression(C=1e5)
#logistic.fit(x_train_lo, y_train_lo)
#print("Classification accuracy:",logistic.score(x_test_lo,y_test_lo))

In [ ]:
#from sklearn.model_selection import cross_val_score
#cross_val_score(logistic, x_train_lo, y_train_lo, cv=5, scoring="accuracy")

In [ ]:
#from sklearn.model_selection import cross_val_predict
#y_scores = cross_val_predict(logistic, x_train_lo, y_train_lo, cv=3,
#method="decision_function")method="decision_function")

In [ ]:
#from sklearn.metrics import precision_recall_curve
#precisions, recalls, thresholds = precision_recall_curve(y_train_lo, y_scores)

In [ ]:
#def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
 #   plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
 #   plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
 #   [...] # highlight the threshold, add the legend, axis label and grid

#plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
#plt.show()

In [ ]:
#from sklearn.metrics import roc_curve
#fpr, tpr, thresholds = roc_curve(y_train_lo, y_scores)

#def plot_roc_curve(fpr, tpr, label=None):
 #   plt.plot(fpr, tpr, linewidth=2, label=label)
  #  plt.plot([0, 1], [0, 1], 'k--') # dashed diagonal
   # [...] # Add axis labels and grid
#plot_roc_curve(fpr, tpr)
#plt.show()

In [ ]:
#from sklearn.metrics import confusion_matrix
#y_test_pred = logistic.predict(x_test_lo)
#confusion_matrix(y_test_lo, y_test_pred)

In [ ]:
#from mlxtend.plotting import plot_confusion_matrix
#import matplotlib.pyplot as plt
#import numpy as np
#fig, ax = plot_confusion_matrix(conf_mat=confusion_matrix(y_test_lo, y_test_pred))
#plt.show()

82% ! A really good result! But is it? s

In [ ]:
#from sklearn import neighbors

#neigh_class = neighbors.KNeighborsClassifier(n_neighbors=5, weights="distance")
#neigh_class.fit(x_train_lo, y_train_lo)
#print("Classification accuracy:",neigh_class.score(x_test_lo,y_test_lo))

In [ ]:
# = neigh_class.predict(x_test_lo)
#fig, ax = plot_confusion_matrix(conf_mat=confusion_matrix(y_test_lo, y_test_pred))
#plt.show()

In [ ]:
#from sklearn import svm

#svc = svm.SVC(kernel='linear', C = 1, gamma='auto')
#svc.fit(x_train_lo, y_train_lo)    
#print("Classification accuracy (linear):", svc.score(x_test_lo, y_test_lo))






In [ ]:
#y_test_pred = svc.predict(x_test_lo)
#fig, ax = plot_confusion_matrix(conf_mat=confusion_matrix(y_test_lo, y_test_pred))
#plt.show()

In [ ]:
#svc = svm.SVC(kernel='poly', degree=3, gamma='auto')
#svc.fit(x_train_lo, y_train_lo)    
#print("Classification accuracy (poly):", svc.score(x_train_lo, y_test_lo))

In [ ]:
#svc = svm.SVC(kernel='rbf',C=1.0, gamma='auto')
#svc.fit(x_train_lo, y_train_lo)    
#print("Classification accuracy (rbf):", svc.score(x_test_lo, y_test_lo))

# Step 6: Loading and resizing of images
In this step images will be loaded into the column named image from the image path from the image folder. We also resize the images as the original dimension of images are 450 x 600 x3 which TensorFlow can't handle, so that's why we resize it into 100 x 75. As this step resize all the 10015 images dimensions into 100x 75 so be patient it will take some time.

In [ ]:
#melanocytic_sample=skin_df2[skin_df2['cell_type']=='Melanocytic nevi'].sample(1000)

In [ ]:
#skin_df3=skin_df2[skin_df2['cell_type']!='Melanocytic nevi']

In [ ]:
#skin_df4 = pd.concat([skin_df3, melanocytic_sample])

In [ ]:
#, ax1 = plt.subplots(1, 1, figsize= (10, 5))#
#skin_df4['cell_type'].value_counts().plot(kind='bar', ax=ax1)

In [ ]:
skin_df['image'] = skin_df['path'].map(lambda x: np.asarray(Image.open(x).resize((100,75))))

In [ ]:
#skin_df4.head()

**As we can see image column has been added in its color format code** 

Most interesting part its always better to see sample of images
Below we will show images of each cancer type

In [ ]:
# Checking the image size distribution
skin_df['image'].map(lambda x: x.shape).value_counts()

### Average color informations


In [ ]:
rgb_info_df = skin_df.apply(lambda x: pd.Series({'{}_mean'.format(k): v for k, v in 
                                  zip(['Red', 'Green', 'Blue'], 
                                      np.mean(x['image'], (0, 1)))}),1)
gray_col_vec = rgb_info_df.apply(lambda x: np.mean(x), 1)
for c_col in rgb_info_df.columns:
    rgb_info_df[c_col] = rgb_info_df[c_col]/gray_col_vec
rgb_info_df['Gray_mean'] = gray_col_vec
rgb_info_df.sample(3)

In [ ]:
for c_col in rgb_info_df.columns:
    skin_df[c_col] = rgb_info_df[c_col].values # we cant afford a copy
    
sns.pairplot(skin_df[['Red_mean', 'Green_mean', 'Blue_mean', 'Gray_mean', 'cell_type']], 
             hue='cell_type', plot_kws = {'alpha': 0.5})

# Step 7 : Train Test Split
In this step we have splitted the dataset into training and testing set of 80:20 ratio

In [ ]:
features=skin_df.drop(columns=['cell_type_idx'],axis=1)
target=skin_df['cell_type_idx']
features.head()
target.head()

In [ ]:
x_train_o, x_test_o, y_train_o, y_test_o = train_test_split(features, target, test_size=0.20,random_state=2)

# Step 8 : Normalization

I choosed to normalize the x_train, x_test by substracting from theor mean values and then dividing by thier standard deviation.

In [ ]:
x_train = np.asarray(x_train_o['image'].tolist())
x_test = np.asarray(x_test_o['image'].tolist())



# Step 9 : Label Encoding
Labels are 7 different classes of skin cancer types from 0 to 6. We need to encode these lables to one hot vectors 

In [ ]:
# Perform one-hot encoding on the labels
y_train = to_categorical(y_train_o, num_classes = 7)
y_test = to_categorical(y_test_o, num_classes = 7)




In [ ]:
x_train

# Step 10 : Splitting training and validation split
I choosed to split the train set in two parts : a small fraction (10%) became the validation set which the model is evaluated and the rest (90%) is used to train the model.

In [ ]:
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.2, random_state = 2)

In [ ]:
# Reshape image in 3 dimensions (height = 75px, width = 100px , canal = 3)
x_train = x_train.reshape(x_train.shape[0], *(75, 100, 3))
x_test = x_test.reshape(x_test.shape[0], *(75, 100, 3))
x_validate = x_validate.reshape(x_validate.shape[0], *(75, 100, 3))

# Step 11.1: Model Building 
# CNN
I used the Keras Sequential API, where you have just to add one layer at a time, starting from the input.

The first is the convolutional (Conv2D) layer. It is like a set of learnable filters. I choosed to set 32 filters for the two firsts conv2D layers and 64 filters for the two last ones. Each filter transforms a part of the image (defined by the kernel size) using the kernel filter. The kernel filter matrix is applied on the whole image. Filters can be seen as a transformation of the image.

The CNN can isolate features that are useful everywhere from these transformed images (feature maps).

The second important layer in CNN is the pooling (MaxPool2D) layer. This layer simply acts as a downsampling filter. It looks at the 2 neighboring pixels and picks the maximal value. These are used to reduce computational cost, and to some extent also reduce overfitting. We have to choose the pooling size (i.e the area size pooled each time) more the pooling dimension is high, more the downsampling is important.

Combining convolutional and pooling layers, CNN are able to combine local features and learn more global features of the image.

Dropout is a regularization method, where a proportion of nodes in the layer are randomly ignored (setting their wieghts to zero) for each training sample. This drops randomly a propotion of the network and forces the network to learn features in a distributed way. This technique also improves generalization and reduces the overfitting.

'relu' is the rectifier (activation function max(0,x). The rectifier activation function is used to add non linearity to the network.

The Flatten layer is use to convert the final feature maps into a one single 1D vector. This flattening step is needed so that you can make use of fully connected layers after some convolutional/maxpool layers. It combines all the found local features of the previous convolutional layers.

In the end i used the features in two fully-connected (Dense) layers which is just artificial an neural networks (ANN) classifier. In the last layer(Dense(10,activation="softmax")) the net outputs distribution of probability of each class.

In [ ]:
# Set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out
input_shape = (75, 100, 3)
num_classes = 7

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=input_shape))
model.add(Conv2D(32,kernel_size=(3, 3), activation='relu',padding = 'Same',))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

# Step 11.2: Using VGG16

VGG16 is a convolutional neural network model proposed by K. Simonyan and A. Zisserman from the University of Oxford in the paper “Very Deep Convolutional Networks for Large-Scale Image Recognition”. 

In [ ]:
conv_base = VGG19(weights = 'imagenet',
                 include_top = False,
                 input_shape = (75, 100, 3))

In [ ]:
conv_base.summary()

In [ ]:
from keras import models
num_classes = 7
model1 = models.Sequential()
model1.add(conv_base)
model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(num_classes, activation='softmax'))

In [ ]:
model1.summary()

# Step 11.3 
## ResNet50 with pre trained weights

In [ ]:
base_model = ResNet50(weights='imagenet',
                      include_top=False, input_shape=(75,100,3))
model3=Sequential()
model3.add(base_model)
# Freeze the layers except the last 4 layers
model3.add(Dropout(0.40))
model3.add(Flatten())
model3.add(Dense(512,activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(7, activation='softmax'))

In [ ]:
model3.summary()

# Step 12: Setting Optimizer and Annealer

Once our layers are added to the model, we need to set up a score function, a loss function and an optimisation algorithm.
We define the loss function to measure how poorly our model performs on images with known labels. It is the error rate between the oberved labels and the predicted ones. We use a specific form for categorical classifications (>2 classes) called the "categorical_crossentropy".
The most important function is the optimizer. This function will iteratively improve parameters (filters kernel values, weights and bias of neurons ...) in order to minimise the loss.
I choosed Adam optimizer because it combines the advantages of two other extensions of stochastic gradient descent. Specifically:

1. Adaptive Gradient Algorithm (AdaGrad) that maintains a per-parameter learning rate that improves performance on problems with sparse gradients (e.g. natural language and computer vision problems).

2. Root Mean Square Propagation (RMSProp) that also maintains per-parameter learning rates that are adapted based on the average of recent magnitudes of the gradients for the weight (e.g. how quickly it is changing). This means the algorithm does well on online and non-stationary problems (e.g. noisy).

Adam realizes the benefits of both AdaGrad and RMSProp.

Adam is a popular algorithm in the field of deep learning because it achieves good results fast.

The metric function "accuracy" is used is to evaluate the performance our model. This metric function is similar to the loss function, except that the results from the metric evaluation are not used when training the model (only for evaluation).

In [ ]:
# Define the optimizer
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

# Adding a Checkpointer

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint('../input/best.hdf5', verbose=1,save_best_only=True)

In [ ]:
def categorical_focal_loss(gamma=2., alpha=.25):
    """
    Softmax version of focal loss.
           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1
      where m = number of classes, c = class and o = observation
    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy
      gamma -- focusing parameter for modulating factor (1-p)
    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper
    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy
    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """

        # Scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Sum the losses in mini_batch
        return K.sum(loss, axis=1)

    return categorical_focal_loss_fixed

In [ ]:
def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [ ]:
# Compile the model
model.compile(optimizer = optimizer , loss ="categorical_crossentropy", metrics=["accuracy"])
model1.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

model3.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ['accuracy'])

In [ ]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In order to make the optimizer converge faster and closest to the global minimum of the loss function, i used an annealing method of the learning rate (LR).

The LR is the step by which the optimizer walks through the 'loss landscape'. The higher LR, the bigger are the steps and the quicker is the convergence. However the sampling is very poor with an high LR and the optimizer could probably fall into a local minima.

Its better to have a decreasing learning rate during the training to reach efficiently the global minimum of the loss function.

To keep the advantage of the fast computation time with a high LR, i decreased the LR dynamically every X steps (epochs) depending if it is necessary (when accuracy is not improved).

With the ReduceLROnPlateau function from Keras.callbacks, i choose to reduce the LR by half if the accuracy is not improved after 3 epochs.

# Data Augmentation
It is the optional step. In order to avoid overfitting problem, we need to expand artificially our HAM 10000 dataset. We can make your existing dataset even larger. The idea is to alter the training data with small transformations to reproduce the variations 

Approaches that alter the training data in ways that change the array representation while keeping the label the same are known as data augmentation techniques. Some popular augmentations people use are grayscales, horizontal flips, vertical flips, random crops, color jitters, translations, rotations, and much more.

By applying just a couple of these transformations to our training data, we can easily double or triple the number of training examples and create a very robust model.

In [ ]:
# With data augmentation to prevent overfitting 

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True # randomly flip images 
            )

datagen.fit(x_train)




In [ ]:
 datagen2 = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
    
datagen2.fit(x_train)

In [ ]:
for X_batch, y_batch in datagen2.flow(x_train, y_train, batch_size=9):
# grid of 3x3 images
    for i in range(0, 9):
        plt.subplot(330 + 1 + i)
        plt.imshow(X_batch[i])

    plt.show()
    break

For the data augmentation, i choosed to :
Randomly rotate some training images by 10 degrees Randomly Zoom by 10% some training images Randomly shift images horizontally by 10% of the width Randomly shift images vertically by 10% of the height 
Once our model is ready, we fit the training dataset .

# Step 13: Fitting the model
In this step finally I fit the model into x_train, y_train. In this step I have choosen batch size of 10 and 50 epochs   as small as your batch size will be more efficiently your model will train and I have choosen 50 epochs to give the model sufficient epochs to train 

## 13.1 NO NORMALIZATION, FOCAL LOSS 75x100 NO CLASS WEIGTHS

In [ ]:
class_weights={
    0: 3.0, # akiec
    1: 1.0, # bcc
    2: 1.0, # bkl
    3: 3.0, # df
    4: 3.0, # mel # Try to make the model more sensitive to Melanoma.
    5: 2.0, # nv
    6: 1.0, # vasc
}

In [ ]:
# Fit the model
epochs = 30
batch_size = 10
history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_validate,y_validate),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

In [ ]:

history1 = model1.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                      epochs = epochs, validation_data = (x_validate,y_validate),
                      verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size,
                      callbacks=[learning_rate_reduction])

In [ ]:

history3 = model3.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_validate,y_validate),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

Come si può leggere dal titolo del paragrafo qua ho utilizzato una loss-function diversa, la **focal loss** che in teoria dovrebbe essere in grado di gestire i campioni sbilanciati in quanto mano a mano che l'esecuzione procede attribuisce meno peso alle classi che identifica correttamente, se non ho capito male. L'ultimo modello sembra essere quello che performa meglio, con picchi intorno all'80% sul validation set. Nella prossima sezione riprovo a lanciare gli stessi modelli, solo provando a normalizzare gli input data. 



# Step 14: Model Evaluation 
In this step we will check the testing accuracy and validation accuracy of our model,plot confusion matrix and also check the missclassified images count of each type

## EVALUATION DEI PRIMI MODELLI DELLA SEZIONE PRECEDENTE

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = model.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
model.save("model.h5")


In [ ]:
loss, accuracy = model1.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = model1.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
model.save("model.h5")

In [ ]:
loss, accuracy = model3.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = model3.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
model.save("model.h5")

In [ ]:
plot_model_history(history)

In [ ]:
plot_model_history(history1)

In [ ]:
plot_model_history(history3)

In [ ]:
# Function to plot confusion matrix    
def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = np.round_(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], 2)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
Y_pred = model3.predict(x_validate)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_validate,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
confusion_mtx = np.round_(confusion_mtx / confusion_mtx.astype(np.float).sum(axis=1),2)

#plt.imshow(confusion_mtx)
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(7)) 

In [ ]:
# Predict the values from the validation dataset
Y_pred = model1.predict(x_validate)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_validate,axis = 1) 
# compute the confusion matrix
confusion_mtx1 = confusion_matrix(Y_true, Y_pred_classes)
confusion_mtx1 = np.round_(confusion_mtx1 / confusion_mtx1.astype(np.float).sum(axis=1),2)
 

# plot the confusion matrix
plot_confusion_matrix(confusion_mtx1, classes = range(7))

In [ ]:
# Predict the values from the validation dataset
Y_pred = model3.predict(x_validate)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_validate,axis = 1) 
# compute the confusion matrix
confusion_mtx3 = confusion_matrix(Y_true, Y_pred_classes)
confusion_mtx3 = np.round_(confusion_mtx3 / confusion_mtx3.astype(np.float).sum(axis=1),2)
 

# plot the confusion matrix
plot_confusion_matrix(confusion_mtx3, classes = range(7))

Now, lets which category has much incorrect predictions

In [ ]:
label_frac_error = 1 - np.diag(confusion_mtx) / np.sum(confusion_mtx, axis=1)
plt.bar(np.arange(7),label_frac_error)
plt.xlabel('True Label')
plt.ylabel('Fraction classified incorrectly')

In [ ]:
label_frac_error = 1 - np.diag(confusion_mtx1) / np.sum(confusion_mtx1, axis=1)
plt.bar(np.arange(7),label_frac_error)
plt.xlabel('True Label')
plt.ylabel('Fraction classified incorrectly')

In [ ]:
label_frac_error = 1 - np.diag(confusion_mtx3) / np.sum(confusion_mtx3, axis=1)
plt.bar(np.arange(7),label_frac_error)
plt.xlabel('True Label')
plt.ylabel('Fraction classified incorrectly')

# NORMALIZATION E TUTTO IL RESTO COME SOPRA


In [ ]:
for image in rangen(x_train):
    

In [ ]:
model.compile(optimizer = optimizer , loss =[categorical_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"])
model1.compile(optimizer = optimizer , loss = [categorical_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"])
#model2.compile(optimizer = optimizer , loss = [categorical_focal_loss(alpha=.25, gamma=2)], metrics = ['accuracy'])
model3.compile(optimizer = optimizer, loss = [categorical_focal_loss(alpha=.25, gamma=2)], metrics = ['accuracy'])

In [ ]:

history_ = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_validate,y_validate),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

In [ ]:
history1_2 = model1.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_validate,y_validate),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

In [ ]:
history3_2 = model3.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data =(x_validate,y_validate),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

Normalizzare sembra dare risultati peggiori rispetto a quelli ottenuti con le immagini normali. Che sia forse dovuto al fatto che in seguito alla normalizzazione i colori delle varie lesioni non si riescano più a distinguere? (Dalle immagini diventa tutto rosse / tuttno nero) In più non riesco a capire coem riesca ad ottenere risultati molto più alti nel training set ma faccia più schifo nel validation. boh, dovremo aspettare la prova del test set

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = model.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
model.save("model.h5")

In [ ]:
loss, accuracy = model1.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = model1.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
model.save("model.h5")

In [ ]:
loss, accuracy = model3.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = model3.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
model.save("model.h5")

In [ ]:
plot_model_history(history_)

In [ ]:
plot_model_history(history1_2)

In [ ]:
plot_model_history(history3_2)

In [ ]:
from sklearn.metrics import *

In [ ]:
# Predict the values from the validation dataset
Y_pred = model.predict(x_validate)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_validate,axis = 1) 
# compute the confusion matrix
confusion_mtx = recall_score(Y_true, Y_pred_classes, average = 'macro')

confusion_mtx = np.round_(confusion_mtx / confusion_mtx.astype(np.float).sum(axis=1),2)

# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(7)) 

In [ ]:
# Predict the values from the validation dataset
Y_pred = model1.predict(x_validate)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_validate,axis = 1) 
# compute the confusion matrix
confusion_mtx1 = confusion_matrix(Y_true, Y_pred_classes)
confusion_mtx1 = np.round_(confusion_mtx1 / confusion_mtx1.astype(np.float).sum(axis=1),2)
 

# plot the confusion matrix
plot_confusion_matrix(confusion_mtx1, classes = range(7)) 

In [ ]:
# Predict the values from the validation dataset
Y_pred = model3.predict(x_validate)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_validate,axis = 1) 
# compute the confusion matrix
confusion_mtx3 = confusion_matrix(Y_true, Y_pred_classes)
confusion_mtx3 = np.round_(confusion_mtx3 / confusion_mtx3.astype(np.float).sum(axis=1),2)
 

# plot the confusion matrix
plot_confusion_matrix(confusion_mtx3, classes = range(7)) 

# STEP NORMALIZATION SOFTMAX CROSS ENTROPY

In [ ]:
#model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
#model1.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
#model2.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics = ['accuracy'])
#model3.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ['accuracy'])

In [ ]:

#history_3 = model.fit_generator(datagen2.flow(x_train,y_train, batch_size=batch_size),
#                              epochs = epochs, validation_data = (x_validate,y_validate),
#                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size
 #                             , callbacks=[learning_rate_reduction])

In [ ]:
#history1_3 = model1.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),à
#                              epochs = epochs, validation_data = datagen.flow(x_validate,y_validate),
 #                             validation_steps=10 ,verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size
  #                            , callbacks=[learning_rate_reduction])

In [ ]:
#history3_3 = model3.fit_generator(datagen2.flow(x_train,y_train, batch_size=batch_size),
#                              epochs = epochs, validation_data = (x_validate,y_validate),
#                             verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size
 #                             , callbacks=[learning_rate_reduction])

In [ ]:
#loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
#loss_v, accuracy_v = model.evaluate(x_validate, y_validate, verbose=1)
#print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
#print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
#model.save("model.h5")

In [ ]:
#loss, accuracy = model1.evaluate(x_test, y_test, verbose=1)
#loss_v, accuracy_v = model1.evaluate(x_validate, y_validate, verbose=1)
#print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
#print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
#model.save("model.h5")

In [ ]:
#loss, accuracy = model3.evaluate(x_test, y_test, verbose=1)#
#loss_v, accuracy_v = model3.evaluate(x_validate, y_validate, verbose=1)
#print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
#print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
#model.save("model.h5")

In [ ]:
#plot_model_history(history_3)

In [ ]:
#plot_model_history(history1_3)

In [ ]:
#plot_model_history(history3_3)

In [ ]:
# Predict the values from the validation dataset
#Y_pred = model.predict(x_validate)
# Convert predictions classes to one hot vectors 
#Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
#Y_true = np.argmax(y_validate,axis = 1) 
# compute the confusion matrix
#confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)

 

# plot the confusion matrix
#plot_confusion_matrix(confusion_mtx, classes = range(7)) 

In [ ]:
# Predict the values from the validation dataset
#Y_pred = model1.predict(x_validate)
# Convert predictions classes to one hot vectors 
#Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
#Y_true = np.argmax(y_validate,axis = 1) 
# compute the confusion matrix
#confusion_mtx1 = confusion_matrix(Y_true, Y_pred_classes)

 

# plot the confusion matrix
#plot_confusion_matrix(confusion_mtx1, classes = range(7)) 

In [ ]:
# Predict the values from the validation dataset
#Y_pred = model3.predict(x_validate)
# Convert predictions classes to one hot vectors 
#Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
#Y_true = np.argmax(y_validate,axis = 1) 
# compute the confusion matrix
#confusion_mtx3 = confusion_matrix(Y_true, Y_pred_classes)

 

# plot the confusion matrix
#plot_confusion_matrix(confusion_mtx3, classes = range(7)) 

In [ ]:
# Compile the model
#model.compile(optimizer = optimizer , loss =[categorical_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"])
#model1.compile(optimizer = optimizer , loss = [categorical_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"])
#model2.compile(optimizer = optimizer , loss = [categorical_focal_loss(alpha=.25, gamma=2)], metrics = ['accuracy'])
#model3.compile(optimizer = optimizer, loss = [categorical_focal_loss(alpha=.25, gamma=2)], metrics = ['accuracy'])

In [ ]:
#history_4 = model.fit_generator(datagen2.flow(x_train,y_train, batch_size=batch_size),à
#                              epochs = epochs, validation_data = (x_validate,y_validate),
#                             verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size
#                              , callbacks=[learning_rate_reduction])

In [ ]:
#history3_4 = model3.fit_generator(datagen2.flow(x_train,y_train, batch_size=batch_size),#
#                              epochs = epochs, validation_data = (x_validate,y_validate),
#                             verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size
 #                             , callbacks=[learning_rate_reduction])

In [ ]:
#plot_model_history(history_4)

In [ ]:
# Predict the values from the validation dataset
# = model3.predict(x_validate)
# Convert predictions classes to one hot vectors 
#Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
#Y_true = np.argmax(y_validate,axis = 1) 
# compute the confusion matrix
#confusion_mtx3 = confusion_matrix(Y_true, Y_pred_classes)

 

# plot the confusion matrix
#plot_confusion_matrix(confusion_mtx3, classes = range(7)) 

# Conclusion
It seems our model has maximum number of incorrect predictions for Basal cell carcinoma which has code 3, then second most missclassified type is Vascular lesions code 5 then Melanocytic nevi code  0 where as Actinic keratoses code 4 has least misclassified type.

We can also further tune our model to easily achieve the accuracy above 80% and I think still this model is efficient in comparison to detection with human eyes having 77.0344% accuracy 

I hope kagglers like my stepwise approach to classify cancer types. If like then kindly dont forget to hit the **like**
